### Jupyter Notebook Description

#### Overview
This Jupyter Notebook addresses a predictive modeling competition involving crime data from San Francisco spanning nearly 12 years. The dataset encompasses various neighborhoods, and the goal is to predict the category of crimes based on time and location information.

#### Context
From 1934 to 1963, San Francisco was notorious for housing some of the world's most infamous criminals on Alcatraz Island. Today, the city is recognized more for its technology industry than its historical criminal associations. However, despite its modern identity, issues such as wealth inequality, housing shortages, and a notable crime rate persist, making this predictive modeling challenge relevant and significant.

#### Dataset
The dataset used in this competition comprises comprehensive crime reports spanning different neighborhoods of San Francisco. It covers a wide range of crime categories and includes temporal and spatial attributes crucial for predictive analysis.

#### Objective
The primary objective of this notebook is to develop machine learning models that accurately predict the category of crimes based on the provided dataset. Evaluation of model performance is based on the multi-class logarithmic loss metric, emphasizing the importance of both precision and reliability in predictions.

#### Methodology
1. **Data Exploration**: 
   - Exploring the structure and attributes of the dataset.
   - Analyzing trends and patterns in crime occurrences across different neighborhoods and time periods.

2. **Feature Engineering**:
   - Preparing the data for modeling by selecting relevant features.
   - Encoding categorical variables and handling missing data if necessary.

3. **Model Development**:
   - Implementing various machine learning algorithms suitable for multi-class classification tasks.
   - Fine-tuning models and optimizing hyperparameters to enhance predictive accuracy.

4. **Evaluation**:
   - Assessing model performance using cross-validation techniques.
   - Optimizing models based on the multi-class logarithmic loss metric to ensure robustness and reliability.

#### Conclusion
By leveraging machine learning techniques on the San Francisco crime dataset, this notebook aims to contribute insights into crime patterns and improve predictive capabilities. The ultimate goal is to develop models that can assist in allocating resources effectively and enhancing public safety measures across the city's diverse neighborhoods.

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns

In [97]:
sns.set_palette(sns.color_palette("bright", 10))

In [131]:
df = pd.read_csv("../data/San Francisco Crime Classification/train.csv", parse_dates=["Dates"])[50000:]

In [132]:
def add_dates(df):
    df["Year"] = df["Dates"].dt.year
    df["Month"] = df["Dates"].dt.month
    df["Day"] = df["Dates"].dt.day

    df.drop("Dates", axis=1, inplace=True)
    return df

In [133]:
df = add_dates(df)

In [134]:
df.head()

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Day
50000,NON-CRIMINAL,AIDED CASE,Tuesday,TARAVAL,NONE,1700 Block of OCEAN AV,-122.460758,37.724771,2014,9,16
50001,VEHICLE THEFT,STOLEN TRUCK,Tuesday,MISSION,NONE,300 Block of UTAH ST,-122.406559,37.765178,2014,9,16
50002,NON-CRIMINAL,CASE CLOSURE,Tuesday,CENTRAL,"ARREST, BOOKED",100 Block of SUTTER ST,-122.403366,37.789945,2014,9,16
50003,LARCENY/THEFT,GRAND THEFT FROM PERSON,Tuesday,SOUTHERN,"ARREST, BOOKED",1100 Block of MARKET ST,-122.413711,37.779666,2014,9,16
50004,OTHER OFFENSES,PROBATION VIOLATION,Tuesday,SOUTHERN,"ARREST, BOOKED",1100 Block of MARKET ST,-122.413711,37.779666,2014,9,16


In [83]:
df.isna().sum()

Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
Year          0
Month         0
Day           0
dtype: int64

In [84]:
df.dtypes

Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
Year            int32
Month           int32
Day             int32
dtype: object

In [33]:
cat_labels = [c for c in df if not pd.api.types.is_numeric_dtype(df[c]) and c != "Category"]

In [34]:
X = df.drop("Category", axis=1)
y = df["Category"]

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

In [36]:
preprocessor = ColumnTransformer([
    ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_labels)
], remainder="passthrough")

In [37]:
X_train_prep = preprocessor.fit_transform(X_train)
X_valid_prep = preprocessor.transform(X_valid)

In [38]:
enc = LabelEncoder()

y_train_encoded = enc.fit_transform(y_train)
y_valid_encoded = enc.transform(y_valid)

In [39]:
model = CatBoostClassifier(verbose=False)

In [40]:
%%time
model.fit(X_train_prep, y_train_encoded, plot=True, eval_set=(X_valid_prep, y_valid_encoded), early_stopping_rounds=5);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: total: 25min 22s
Wall time: 7min 19s


In [41]:
model.score(X_valid_prep, y_valid_encoded)

0.9854